# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902081


The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.

In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/98 [00:00<?, ?it/s]

Rendering models:   2%|▏         | 2/98 [00:02<02:20,  1.46s/it]

Rendering models:   3%|▎         | 3/98 [00:05<02:58,  1.87s/it]

Rendering models:   4%|▍         | 4/98 [00:05<02:07,  1.35s/it]

Rendering models:   5%|▌         | 5/98 [00:06<01:32,  1.01it/s]

...

Rendering models:  95%|█████████▍| 93/98 [00:20<00:00,  5.55it/s]

Rendering models:  96%|█████████▌| 94/98 [00:20<00:00,  6.00it/s]

Rendering models:  98%|█████████▊| 96/98 [00:20<00:00,  6.61it/s]

Rendering models: 100%|██████████| 98/98 [00:21<00:00,  6.84it/s]

kayleebug2017                         0.001192
equidad1                              0.115000
not-logged-in-587a4e9ad373b8ff5983    0.000501
Thedunkmasta                          0.002865
Thedunkmasta                          0.001094
Thedunkmasta                          0.003193
Thedunkmasta                          0.002366
Thedunkmasta                          0.002922
Thedunkmasta                          0.002102
Thedunkmasta                          0.002041
not-logged-in-7526601586eaa938b3ee    0.000991
Thedunkmasta                          0.002052
mlang2017                             0.004587
ElisabethB                            0.143156
not-logged-in-f696328c08d99f1bed52    0.039911
Linda_J._Berkel                       0.000158
Jeff_H1117                            0.000665
next_ghost                            0.000296
5404                                  0.004036
GabeGramy                             0.073031
msalin03                              0.000235
clizardo     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())